In [1]:
import requests
import altair as alt
import pandas as pd
from pandas.io.json import json_normalize

from pyvis.network import Network
from ipywidgets import interact

import re

%load_ext autoreload
%autoreload 2
# import visualizations1 as viz
%cd ../intervals
from main import *
from main_objs import *
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## Relationships heatmaps

In [2]:
df_relationships = pd.read_csv('../CRIM_Intervals_Notebooks/relationships.csv')
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_0011 = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[0]].copy()

In [3]:
df_relationships_0011.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
0,0,http://crimproject.org/data/relationships/1/,1,Quotation,Fuga,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...",http://crimproject.org/data/observations/2/,2,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6/1,1,1+3,1+3,3,3/@1-4,@1-3,@1-3+@1-4,@1-3+@..."
1,1,http://crimproject.org/data/relationships/2/,2,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/4/,4,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-8/1,1+4,1+4,4/@3,@1-4+@4,@1+@1-3,@1"
2,2,http://crimproject.org/data/relationships/3/,3,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9,14-18/1,1,1-2,1-2,1-2,3,3,3-4,3-4,3-4/@3,@...",http://crimproject.org/data/observations/6/,6,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_5,Missa Tota pulchra es: Agnus Dei,"9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1"
3,3,http://crimproject.org/data/relationships/4/,4,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/8/,8,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-10/1,1+4,1+4,1+4,1+4,4/@3,@1-4+@4,@1-4.5+@1-..."
183,183,http://crimproject.org/data/relationships/184/,184,Non-mechanical transformation,Imitative duo,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-7,10-16/1,1,1-2,1-2,2,2,2,3,3,3-4,3-4,4,4,4/...",http://crimproject.org/data/observations/368/,368,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6,10-13/1,1,1+3,1+3,3,3,2,2-3,2-3,3/@1-4,@1-..."


In [4]:
df_relationships_0011[df_relationships_0011['relationship_type']== "Quotation, Mechanical transformation"]

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
256,256,http://crimproject.org/data/relationships/258/,258,"Quotation, Mechanical transformation",Fuga,True,False,True,True,True,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"76-77/1+3,1+3/@3-4.5+@4,@1-2+@1-2.5",http://crimproject.org/data/observations/516/,516,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"85-87/1,1-2,2/@4,@1-3+@2-4.5,@1"


In [5]:
df_relationships_0011.iloc[0, :]

Unnamed: 0                                                                                 0
url                                             http://crimproject.org/data/relationships/1/
id                                                                                         1
relationship_type                                                                  Quotation
musical_type                                                                            Fuga
rt_q                                                                                    True
rt_q_x                                                                                  True
rt_q_monnayage                                                                         False
rt_tm                                                                                  False
rt_tm_snd                                                                              False
rt_tm_minv                                                            

In [6]:
viz.plot_relationship_heatmap(df_relationships_0011, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

In [7]:
# df_relationships.iloc[3, :]

In [8]:
# def preprocess_df(df, ema_column, ):
#     # preprocess the df to NOT have multiple ema/relationships on the same line
#     df = df.copy()
#     df 

def retrieve_notes_and_durations(piece_url, ema):
    # get the piece
    corpus = CorpusBase([piece_url])
    # get the notes and rests
    model = corpus.scores[0]
    nr = model.getNoteRest()
    dur = model.getDuration()
    beat = model.getBeat()
    
    notes_table = pd.concat([nr, dur, beat], axis=1)
    

def calculate_similarity(df, melody_weight=0.5):
    """
    return a new column with the similarity score for the 
    observations
    by default, we evaluated the melody and the duration equaly (both at 0.5).
    however, the users can choose how heavy they want to weight melody in the caclulation
    """
    
    # retrieve notes and duration from ema
    pass

In [9]:
#retrieve_notes_and_durations('https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/CRIM_Model_0011.mei', '1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,@1-3,@1')

## Observations heatmap

In [10]:
df_observations = pd.read_csv('../CRIM_Intervals_Notebooks/observations.csv')

In [11]:
df_observations_pen = df_observations[df_observations['musical_type']=='Periodic entry'].copy()
df_observations_pen.iloc[0, :]

Unnamed: 0                                                          5
url                       http://crimproject.org/data/observations/6/
id                                                                  6
ema                     9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1
musical_type                                           Periodic entry
                                          ...                        
observer.name                                            Drew Ivarson
piece.url           http://crimproject.org/data/pieces/CRIM_Mass_0...
piece.piece_id                                       CRIM_Mass_0009_5
piece.full_title                     Missa Tota pulchra es: Agnus Dei
piece.mass                                             CRIM_Mass_0009
Name: 5, Length: 92, dtype: object

In [12]:
def add_mel_int_to_graph(graph, melodic_interval):
    # add relationships
    melodic_interval = melodic_interval.astype(str)
    
    for node in melodic_interval:
        if node in peaks:
            graph.add_node(node, color='red')
        else:
            graph.add_node(node)
    graph.add_node('start!', color='red')
  
    melodic_interval = set(melodic_interval)
    for node in melodic_interval:
        prev_node = re.sub(r'([+-])(?!$)', r'\1,', node).split(",")[:-1]
        if not prev_node:
            graph.add_edge('start!', node)
        else:
            prev_node = "".join(prev_node)
            if prev_node in melodic_interval:
                graph.add_edge(prev_node, node)
    

In [13]:
# test = Network(notebook=True, directed=True)
# add_mel_int_to_graph(test, df_observations_pen['mt_pe_int'],   
#                      peaks = {'1+1+',
#                              '1+8+',
#                              '1+8-',
#                              '4-5-',
#                              '5+5+',
#                              '5-8+',
#                              '5-8-',
#                              '8+12-',
#                              '8+8-',
#                              '8-1+',
#                              '8-12+',
#                              '8-5+',
#                              '8-8+'})

In [14]:
# test.show('../CRIM_Intervals_Notebooks/mel.html')

In [15]:
# test1 = Network(notebook=True, directed=True)
# # df_observations_fuga = df_observations[df_observations['musical_type']=='Fuga'].copy()
# add_mel_int_to_graph(test1, df_observations_fuga['mt_fg_int'], {'5+8-7+4-5+', '5-8+12-8+8+', '5-9+4+', 'start!'})

In [16]:
# edges = test1.get_edges()
# edges
# peaks = set(edge['from'] for edge in edges)
# remove_peaks = set(edge['to'] for edge in edges)

# peaks = peaks - remove_peaks
# peaks

In [17]:
# test1.show('../CRIM_Intervals_Notebooks/mel1.html')

In [18]:
# net = Network(notebook=True, directed=True)
# net.add_node(1, group='odd')
# net.add_node(2, group='even')
# net.add_node(3, group='odd')
# net.add_node(4, group='even')
# net.add_node(5, group='odd')
# net.add_node(7, group='odd')
# net.show('../CRIM_Intervals_Notebooks/haha.html')

In [19]:
df_obvservations_0011 = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0011'].copy()
df_observations['mt_fg_tint'].value_counts()

S1                    146
M6                     87
B1                     77
M2                     61
M1                     49
                     ... 
M4/2/12                 1
S2/1/1/3/2              1
M4/4/4/12/5             1
M1/17/6/7/1             1
M1/1/3/1/1/5/4/7/9      1
Name: mt_fg_tint, Length: 570, dtype: int64

In [20]:
net, df = viz.plot_musical_type_network(df_observations, 'pen', pieces=None)

In [21]:
net.show_buttons(filter_=['physics'])

In [22]:
net.show('../CRIM_Intervals_Notebooks/pen.html')

In [23]:
net2, df2 = viz.plot_musical_type_network(df_observations, 'fuga', pieces=None)
# net2.show('../CRIM_Intervals_Notebooks/fuga.html')

In [24]:
net2.show('../CRIM_Intervals_Notebooks/fuga.html')

In [25]:
@interact(search_pattern="", option=['starts with', 'ends with', 'contains'])
# def manipulate_network_df(search_pattern, option='starts with'):
#     if option=='starts with':
#         mask = df['mt_pe_int'].astype(str).str.startswith(pat=search_pattern)
#     elif option=='ends with':
#         mask = df['mt_pe_int'].astype(str).str.endswith(pat=search_pattern)
#     else:
#         mask = df['mt_pe_int'].astype(str).str.contains(pat=search_pattern, regex=False)
#     filtered_df = df[mask].copy()
#     return filtered_df.fillna("-").style.applymap(lambda x: "background: #ccebc5" if search_pattern in x else "")

interactive(children=(Text(value='', description='search_pattern'), Dropdown(description='option', options=('s…

In [26]:
viz.plot_relationship_heatmap(df_obvservations_0011, 'ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

## Ngrams heatmaps

In [27]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=4)
mel = model.getMelodic(compound=False, unit=4)
mel_ngrams = model.getNgrams(df=mel, n=4, cell_type=str)

Requesting file from https://crimproject.org/mei/CRIM_Model_0017.mei...
Successfully imported.


In [28]:
mel_ngrams.stack().dropna().value_counts() 

-P1, -P1, -P1, -P1    43
-P1, -P1, M2, -P1     14
-P1, M2, -P1, -M3     14
-P1, -m2, -M3, M2     10
-P1, M2, m2, -P1       8
                      ..
P8, -m3, m3, M2        1
m2, -m2, -M2, -P1      1
P8, -m3, P4, -M3       1
-M2, P5, -P4, -P1      1
-M3, -m2, m2, M2       1
Length: 481, dtype: int64

In [29]:
mel_ngrams.head()

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,"-P1, -P1, -P1, M2","-P1, M3, -M3, P8",NaN,NaN,NaN,NaN
8.0,"-P1, -P1, M2, -P1","M3, -M3, P8, -M2",NaN,NaN,NaN,NaN
12.0,"-P1, M2, -P1, -P1","-M3, P8, -M2, -m3",NaN,NaN,NaN,NaN
16.0,"M2, -P1, -P1, -P1","P8, -M2, -m3, m3",NaN,NaN,NaN,NaN
20.0,"-P1, -P1, -P1, M2","-M2, -m3, m3, M2",NaN,NaN,NaN,NaN


In [30]:
mel_ngrams_tuple = model.getNgrams(df=mel, n=4, cell_type=tuple)
mel_ngrams_list = model.getNgrams(df=mel, n=4, cell_type=list)

In [31]:
mel_ngrams_tuple.head()

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,"(-P1, -P1, -P1, M2)","(-P1, M3, -M3, P8)",NaN,NaN,NaN,NaN
8.0,"(-P1, -P1, M2, -P1)","(M3, -M3, P8, -M2)",NaN,NaN,NaN,NaN
12.0,"(-P1, M2, -P1, -P1)","(-M3, P8, -M2, -m3)",NaN,NaN,NaN,NaN
16.0,"(M2, -P1, -P1, -P1)","(P8, -M2, -m3, m3)",NaN,NaN,NaN,NaN
20.0,"(-P1, -P1, -P1, M2)","(-M2, -m3, m3, M2)",NaN,NaN,NaN,NaN


In [32]:
viz.plot_close_match_heatmap(mel_ngrams, "-P1, -P1, -P1, M2", selected_patterns=[], voices=[])

alt.Chart(...)

In [33]:
viz.plot_close_match_heatmap(mel_ngrams_tuple, "1, 2, 1, -3", selected_patterns=[], voices=[])

alt.Chart(...)

In [34]:
viz.plot_close_match_heatmap(mel_ngrams_list, "1, -2, -3, 2", selected_patterns=[], voices=[])

alt.Chart(...)

## Heatmaps with for ngrams!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

In [35]:
nr = model.getNoteRest()
dur = model.getDuration(nr)
nr.reindex_like(dur)
nr_ngrams = model.getNgrams(df=nr, n=15)
dur_ngrams = model.getNgrams(df=dur, n=15)

In [36]:
nr_ngrams

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,NaN,"(G3, G3, A3, B3, C4, G3, G4, F4, E4, F4, D4, C...",NaN,NaN,NaN,NaN
4.0,NaN,"(G3, A3, B3, C4, G3, G4, F4, E4, F4, D4, C4, D...",NaN,NaN,NaN,NaN
7.0,NaN,"(A3, B3, C4, G3, G4, F4, E4, F4, D4, C4, D4, E...",NaN,NaN,NaN,NaN
8.0,NaN,"(B3, C4, G3, G4, F4, E4, F4, D4, C4, D4, E4, F...",NaN,NaN,NaN,NaN
10.0,NaN,"(C4, G3, G4, F4, E4, F4, D4, C4, D4, E4, F4, G...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1430.0,NaN,NaN,NaN,NaN,NaN,"(F3, G3, F3, G3, D3, E3, F3, G3, F3, F3, G3, A..."
1432.0,NaN,NaN,NaN,NaN,NaN,"(G3, F3, G3, D3, E3, F3, G3, F3, F3, G3, A3, D..."
1434.0,NaN,NaN,NaN,NaN,NaN,"(F3, G3, D3, E3, F3, G3, F3, F3, G3, A3, D3, F..."
1444.0,NaN,NaN,NaN,"(D4, D4, D3, E3, F3, G3, A3, D3, E3, F3, G3, A...",NaN,NaN


In [37]:
dur_ngrams

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,"(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(4.0, 2.0, 8.0, 8.0, 8.0, 8.0, 4.0, 3.0, 4.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.0, ...","(8.0, 8.0, 8.0, 8.0, 4.0, 2.0, 8.0, 8.0, 4.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 4.0, ..."
4.0,NaN,"(3.0, 1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 2.0, 3.0, ...",NaN,NaN,NaN,NaN
7.0,NaN,"(1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 2.0, 3.0, 1.0, ...",NaN,NaN,NaN,NaN
8.0,"(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(2.0, 8.0, 8.0, 8.0, 8.0, 4.0, 3.0, 4.0, 2.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.0, 2.0, ...","(8.0, 8.0, 8.0, 4.0, 2.0, 8.0, 8.0, 4.0, 4.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 4.0, 2.0, ..."
10.0,NaN,"(2.0, 2.0, 4.0, 1.0, 1.0, 2.0, 3.0, 1.0, 1.0, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1462.0,NaN,NaN,NaN,"(2.0, 16.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.3333333...",NaN,NaN
1463.0,NaN,NaN,"(1.0, 2.0, 1.0, 1.0, 4.0, 2.0, 1.0, 4.0, 1.0, ...",NaN,NaN,"(1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 2.0, 2.0, 4.0, ..."
1464.0,NaN,"(8.0, 3.0, 1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 2.0, ...","(1.0, 8.0, 8.0, 8.0, 4.0, 6.0, 8.0, 4.0, 2.0, ...","(4.0, 1.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...",NaN,"(16.0, 3.0, 2.0, 4.0, 1.0, 1.0, 1.0, 1.0, 3.0,..."
1468.0,NaN,NaN,NaN,"(16.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.333333333333...",NaN,NaN


In [38]:
# convert the patterns to string for better display in the heatmap
nr_ngrams  = nr_ngrams .applymap(lambda cell: ", ".join(item for item in cell) if pd.notna(cell) else cell) 

# convert the tuple to the total duration of corresponding ngrams
dur_ngrams = dur_ngrams.applymap(lambda cell: sum(cell) if pd.notna(cell) else cell)

In [39]:
dur_ngrams 

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,108.0,62.0,112.0,90.000000,85.0,89.0
4.0,NaN,25.0,NaN,NaN,NaN,NaN
7.0,NaN,23.0,NaN,NaN,NaN,NaN
8.0,106.0,66.0,112.0,82.666667,85.0,85.0
10.0,NaN,26.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1462.0,NaN,NaN,NaN,35.333333,NaN,NaN
1463.0,NaN,NaN,28.0,NaN,NaN,41.0
1464.0,NaN,32.0,74.0,33.000000,NaN,46.0
1468.0,NaN,NaN,NaN,34.333333,NaN,NaN


In [40]:
# plot only the most popular patterns
most_popular = nr_ngrams.stack().value_counts().head(20).index.to_list()

In [41]:
viz.plot_ngrams_heatmap(nr_ngrams, dur_ngrams, selected_pattern=most_popular, voices=[])

alt.VConcatChart(...)

In [42]:
# retrieve necessary ngrams
corpus8 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei'])
corpus17 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
corpus11 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0011.mei'])

Requesting file from https://crimproject.org/mei/CRIM_Model_0008.mei...
Successfully imported.
Memoized piece detected...
Requesting file from https://crimproject.org/mei/CRIM_Model_0011.mei...
Successfully imported.


In [43]:
model8 = corpus8.scores[0]
mel8 = model8.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams8 = model8.getNgrams(df=mel8, n=4)

model17 = corpus17.scores[0]
mel17 = model17.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams17 = model17.getNgrams(df=mel17, n=4)

model11 = corpus11.scores[0]
mel11 = model11.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams11 = model11.getNgrams(df=mel11, n=4)

In [44]:
mel_ngrams8.index.name = "offset"

mel_ngrams8.reset_index().melt(id_vars=['offset'], value_name='pattern', var_name=['voice'])

,offset,voice,pattern
0,4.0,[Superius],"(4, 1, 1, 2)"
1,8.0,[Superius],"(1, 1, 2, 2)"
2,12.0,[Superius],"(1, 2, 2, -3)"
3,16.0,[Superius],"(2, 2, -3, 1)"
4,20.0,[Superius],NaN
...,...,...,...
1087,1272.0,Bassus,"(-5, 1, 1, 1)"
1088,1276.0,Bassus,"(1, 1, 1, 1)"
1089,1280.0,Bassus,"(1, 1, 1, 1)"
1090,1284.0,Bassus,"(1, 1, 1, 1)"


In [45]:
# add from one model's ngram to the graph
def add_ngram_to_graph(graph, name, ngrams):
    ngrams.index.name = "offset"
    ngrams = ngrams.reset_index().melt(id_vars=['offset'], value_name='pattern', var_name=['voice']).dropna()
    # TODO fix for loop!
    for i in range(len(ngrams)):
        ngram = ngrams.iloc[i, :]
        # add nodes
        ngram_sliced = [ngram['pattern'][:i] for i in range(1, len(ngram['pattern']) + 1)]
        ngram_sliced = [", ".join(item for item in cell) for cell in ngram_sliced]
        ngram_sliced.append(name + ' ' + str(ngram['offset']))
        for node in ngram_sliced:
            graph.add_node(node)
        # add edges
        for i in range(len(ngram_sliced) - 1):
            graph.add_edge(ngram_sliced[i], ngram_sliced[i+1])

In [46]:
# net = Network(notebook=True, directed=True)
# add_ngram_to_graph(net, 'CRIM_Model_0008.mei', mel_ngrams8)
# add_ngram_to_graph(net, 'CRIM_Model_0017.mei', mel_ngrams17)
# add_ngram_to_graph(net, 'CRIM_Model_0011.mei', mel_ngrams11)

In [47]:
# net.show('../CRIM_Intervals_Notebooks/mygraph.html')